In [ ]:
!pip install -U diffusers
!pip install huggingface_hub


In [ ]:
!huggingface-cli login

In [ ]:
import os
import gc
import pandas as pd
import torch
from tqdm import tqdm
from diffusers import StableDiffusionPipeline


# 1. Configuring the environment and defining paths

csv_path = "mscoco-blip-vit_captions.csv"   # Specifying the path of the captions CSV file
model_name = "SD2"  # Assigning a name to the model for saving outputs
model_id = "stabilityai/stable-diffusion-2-base"  # Specifying the Stable Diffusion 2 model identifier
save_dir = model_name  # Setting the output directory to match the model name
os.makedirs(save_dir, exist_ok=True)  # Creating the output directory if it is not already existing

device = "cuda" if torch.cuda.is_available() else "cpu"  # Detecting and assigning GPU or CPU device


# 2. Loading the Stable Diffusion 2 model

pipe = StableDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16
).to(device)  # Loading the SD2 model with half precision on the selected device

# Enabling memory optimization techniques
pipe.enable_attention_slicing()  # Enabling attention slicing to reduce memory usage
pipe.enable_sequential_cpu_offload()  # Enabling sequential offloading of model parts to CPU when not in use


# 3. Reading the CSV file in chunks while skipping specified rows

chunksize = 50   # Defining how many rows are being processed per chunk
skip_rows = 0   # Defining how many rows are being skipped at the start of the file

print(f"Reading {csv_path} in chunks of {chunksize} rows, skipping first {skip_rows} rows...")

reader = pd.read_csv(csv_path, chunksize=chunksize, skiprows=range(1, skip_rows+1))
# Using range(1, N+1) for ensuring the first N rows are being skipped while keeping the header row intact

for chunk in reader:
    for row in tqdm(chunk.itertuples(), total=len(chunk), desc=f"Generating with {model_name}"):
        image_name = row.image_name
        caption = row.mscoco_caption   # Extracting the caption text for image generation


        # 4. Generating an image from the caption

        image = pipe(caption, height=512, width=512).images[0]


        # 5. Saving the generated image

        save_path = os.path.join(save_dir, f"{model_name}-{image_name}.jpeg")
        image.save(save_path, "JPEG")


        # 6. Releasing memory resources

        del image  # Deleting the image object from memory
        torch.cuda.empty_cache()  # Clearing unused GPU memory
        gc.collect()  # Running garbage collection to free additional memory

print(" All images are being generated and saved successfully!")

import shutil  # Importing library for handling file operations

# Creating a zip archive of the output folder
shutil.make_archive("SD2", 'zip', "SD2")

# Downloading the zipped folder to the local machine
from google.colab import files
files.download("SD2.zip")
